In [1]:
import pandas as pd
import numpy as np
import time

data_dir = "./data/lastfm/lastfm-dataset-360K"

col_names = ["user", "artist-mbid", "artist-name", "total-plays"]
data = pd.read_csv(
    "./data/lastfm/lastfm-dataset-360K/lastfm_small.tsv",
    sep="\t",
    header=None,
    names=col_names
)
data.head()

,user,artist-mbid,artist-name,total-plays
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706


In [2]:
# заполняем пустые значения
data.fillna("None", inplace=True)
# заменим строковые идентификаторы числовыми кодами
# добавляем к индексам единицы, потому что в mrec,
# который будем использовать для оценки качества, индексы начинаются с единицы
data["user_id"] = data["user"].astype("category").cat.codes.copy() + 1
data["artist_id"] = data["artist-mbid"].astype("category").cat.codes.copy() + 1
# убираем лишние колонки
data.drop(["artist-name", "artist-mbid", "user"], axis=1, inplace=True)
data.head()

,total-plays,user_id,artist_id
0,2137,1,15531
1,1099,1,63469
2,897,1,46858
3,717,1,15968
4,706,1,48969


In [11]:
data.user_id.max()

20465

In [12]:
data.artist_id.max()

66799

In [3]:
# разобьём наблюдения на тестовую и обучающую выборки
test_indices = np.random.choice(
    data.index.values,
    replace=False,
    size=int(len(data.index.values) * 0.2)
)
test_data = data.iloc[test_indices]
train_data = data.drop(test_indices)

In [4]:
test_user_set = set(test_data["user_id"].unique())
train_user_set = set(train_data["user_id"].unique())
print("нет в обучающей выборке, но есть в тестовой: {}".format(
    len(test_user_set - train_user_set)))
print("нет в тестовой выборке, но есть в обучающей: {}".format(
    len(train_user_set - test_user_set)))
print("всего пользователей: {}".format(len(data["user_id"].unique())))

нет в обучающей выборке, но есть в тестовой: 0
нет в тестовой выборке, но есть в обучающей: 26
всего пользователей: 20465


In [5]:
# исключим таких пользователей из тестовой и обучающей выборок
user_ids_to_exclude = (test_user_set - train_user_set).union(train_user_set - test_user_set)
bad_indices = test_data[test_data["user_id"].isin(user_ids_to_exclude).values].index
test_data.drop(bad_indices, inplace=True)
bad_indices = train_data[train_data["user_id"].isin(user_ids_to_exclude).values]
train_data.drop(bad_indices.index, inplace=True)

C:\Users\mershov\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [23]:
from os import path
test_file_name = "lastfm.test.0"
test_data[["user_id", "artist_id", "total-plays"]].to_csv(
    path.join(data_dir, test_file_name),
    sep="\t",
    header=False,
    index=False
)
train_file_name = "lastfm.train.0"
train_data[["user_id", "artist_id", "total-plays"]].to_csv(
    data_dir + '/train/' + train_file_name,
    sep="\t",
    header=False,
    index=False
)

In [7]:
# функция, которая красиво печатает информацию о разреженных матрицах
from scipy.sparse import csr_matrix

def sparse_info(sparse_matrix: csr_matrix) -> None:
    print("Размерности матрицы: {}".format(sparse_matrix.shape))
    print("Ненулевых элементов в матрице: {}".format(sparse_matrix.nnz))
    print("Доля ненулевых элементов: {}"
          .format(sparse_matrix.nnz / sparse_matrix.shape[0] / sparse_matrix.shape[1])
    )
    print("Среднее значение ненулевых элементов: {}".format(sparse_matrix.data.mean()))
    print("Максимальное значение ненулевых элементов: {}".format(sparse_matrix.data.max()))
    print("Минимальное значение ненулевых элементов: {}".format(sparse_matrix.data.min()))

In [32]:
# создаём разреженную матрицу item*user
from scipy.sparse import coo_matrix
import numpy as np

plays = coo_matrix((
    train_data["total-plays"].astype(np.float64),
    (
        train_data["artist_id"],
        train_data["user_id"]
    )
))

sparse_info(plays.tocsr())

Размерности матрицы: (66799, 20466)
Ненулевых элементов в матрице: 799771
Доля ненулевых элементов: 0.0005850092345351762
Среднее значение ненулевых элементов: 216.68098368157885
Максимальное значение ненулевых элементов: 135392.0
Минимальное значение ненулевых элементов: 1.0


In [9]:
import implicit

In [10]:
#Обучаем CosineRecommender
from implicit.nearest_neighbours import CosineRecommender
# initialize a model
model = CosineRecommender()
model.fit(plays.tocsr())
sparse_info(model.similarity)

Размерности матрицы: (66799, 66799)
Ненулевых элементов в матрице: 1219207
Доля ненулевых элементов: 0.00027323575245242115
Среднее значение ненулевых элементов: 0.44382339304370244
Максимальное значение ненулевых элементов: 1.0000000579384145
Минимальное значение ненулевых элементов: 0.0


In [26]:
print("получаем рекомендации для всех пользователей по knn")
start = time.time()
user_plays = plays.T.tocsr()
with open('./data/lastfm/lastfm-dataset-360K/knn/' + 'recs/' + test_file_name + '.recs.tsv', "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей
получили рекомендации для всех пользователей за 22.652916431427002 секунд


In [27]:
import implicit

In [33]:
#Обучаем ALS
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares()
print("обучаем ALS")
start = time.time()
model.fit(plays)
print("обучили ALS за {} секунд".format(
        time.time() - start))
#sparse_info(model.similarity)

обучаем ALS
обучили ALS за 14.393741369247437 секунд


In [35]:
print("получаем рекомендации для всех пользователей по als")
start = time.time()
user_plays = plays.T.tocsr()
with open('./data/lastfm/lastfm-dataset-360K/als/' + 'recs/' + test_file_name + '.recs.tsv', "w") as output_file:
    for user_id in test_data["user_id"].unique():
        for artist_id, score in model.recommend(user_id, user_plays):
                output_file.write("%s\t%s\t%s\n" % (user_id, artist_id, score))
print("получили рекомендации для всех пользователей за {} секунд".format(
        time.time() - start))

получаем рекомендации для всех пользователей по als
получили рекомендации для всех пользователей за 116.2594780921936 секунд


# Lets move to Ubuntu!

In [1]:
from os import path, environ

In [2]:
data_dir = '/home/mershov/data/lastfm/lastfm-dataset-360K'
environ["DATA_DIR"] = data_dir

In [3]:
# используем mrec в качестве метрики качества
# оцениваем результат cosine knn
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/knn/recs

[2017-12-08 00:10:17,809] INFO: processing /home/mershov/data/lastfm/lastfm-dataset-360K/lastfm.test.0...
None
mrr            0.0508 +/- 0.0000
prec@5         0.0169 +/- 0.0000
prec@10        0.0151 +/- 0.0000
prec@15        0.0101 +/- 0.0000
prec@20        0.0076 +/- 0.0000


In [4]:
# оцениваем результат als
!mrec_evaluate \
    --input_format=tsv --test_input_format=tsv \
    --train $DATA_DIR/lastfm.test.0 \
    --recsdir $DATA_DIR/als/recs

[2017-12-08 00:11:07,920] INFO: processing /home/mershov/data/lastfm/lastfm-dataset-360K/lastfm.test.0...
None
mrr            0.3172 +/- 0.0000
prec@5         0.1393 +/- 0.0000
prec@10        0.1139 +/- 0.0000
prec@15        0.0759 +/- 0.0000
prec@20        0.0569 +/- 0.0000
